In [ ]:
%pip install datasets==2.16.0 mamba-ssm==1.1.1 accelerate==0.25.0

In [ ]:
# See: https://www.reddit.com/r/LocalLLaMA/comments/18da1al/an_interactive_demo_for_mambachat/
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("nampdn-ai/tinystories-vietnamese", split="train[:1000]")
tokenizer = AutoTokenizer.from_pretrained("VietAI/gpt-neo-1.3B-vietnamese-news")
# dataset = load_dataset("roneneldan/TinyStories", split="train[:1000]")
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token

train_dataset = dataset.map(lambda d: tokenizer(d["vi"]))

In [4]:
from mamba_ssm.models.config_mamba import MambaConfig
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel

config = MambaConfig()
config.d_model = 256
config.n_layer = 16
config.vocab_size = tokenizer.vocab_size
model = MambaLMHeadModel(config, device="cuda")

In [5]:
# See: https://github.com/state-spaces/mamba/blob/1df0df1/benchmarks/benchmark_generation_mamba_simple.py
def generate(input_text: str) -> str:
    input_tokens = tokenizer(input_text, return_tensors="pt")
    input_ids = input_tokens.input_ids.to("cuda")
    output_ids = model.generate(input_ids, max_length=100, return_dict_in_generate=True)
    output_text = tokenizer.batch_decode(output_ids.sequences.tolist())[0]
    return output_text

In [6]:
import os
import torch
from transformers import Trainer, TrainingArguments

# See: https://github.com/havenhq/mamba-chat/blob/bbc9ef6/trainer/mamba_trainer.py
class MambaTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        input_ids = inputs.pop("input_ids")
        lm_logits = model(input_ids).logits
        labels = input_ids.to(lm_logits.device)
        shift_logits = lm_logits[:, :-1, :].contiguous()
        labels = labels[:, 1:].contiguous()
        loss_fct = torch.nn.CrossEntropyLoss()
        lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))
        return lm_loss

    def save_model(self, output_dir, _internal_call):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        torch.save(self.model.state_dict(), f"{output_dir}/pytorch_model.bin")
        self.tokenizer.save_pretrained(output_dir)

trainer = MambaTrainer(
    model=model,
    args=TrainingArguments("mamba-tiny", logging_strategy="epoch", num_train_epochs=20, report_to="none"),
    train_dataset=train_dataset, tokenizer=tokenizer,
)
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
125,7.994200
250,5.720100
375,4.286500
500,3.531400
625,3.168400
750,2.920700
875,2.762900
1000,2.664900
1125,2.533800
1250,2.495300


TrainOutput(global_step=2500, training_loss=2.982362841796875, metrics={'train_runtime': 606.1717, 'train_samples_per_second': 32.994, 'train_steps_per_second': 4.124, 'total_flos': 0.0, 'train_loss': 2.982362841796875, 'epoch': 20.0})

In [7]:
generate("Trời ")

'Trời. Cô bé rất thích chơi với đồ chơi của mình. Một ngày nọ, cô bé nhìn thấy chiếc xe đồ chơi của mình và muốn cho bạn.\nMột ngày nọ, Tim nhìn thấy một con chim nhỏ đến với nó. Con chim nói, "Tôi có thể giúp bạn."\nMột ngày nọ, một con chim nhỏ đến với cô bé đến và nói, "Tôi không thể đi được. Nó nói, "Tôi sẽ đi tìm bạn. Nó không thể tìm thấy nó'

In [8]:
# Examining
sum(p.numel() for p in model.parameters())

22368512